In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint

In [11]:
headers = {"Accept-Language": "en-US, en;q=0.5"}
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []
certificate=[]

In [15]:
pages = np.arange(1, 1001, 50)
for page in pages:
    

    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&start=" + str(page) + "&ref_=adv_nxt", headers=headers)

    soup = BeautifulSoup(page.text, 'html.parser')
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
  
    sleep(randint(2,10))

    for container in movie_div:
        name=container.h3.a.text
        titles.append(name)
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        certificat=container.find('span',class_='certificate').text if container.p.find('span',class_='certificate') else '-'
        certificate.append(certificat)
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        rate=float(container.strong.text)
        imdb_ratings.append(rate)
        meta_scores=container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(meta_scores)
        nv = container.find_all('span', attrs={'name': 'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses) 
        
        


In [16]:
print(titles)
print(years)
print(time)
print(imdb_ratings)
print(metascores)
print(votes)
print(us_gross)
print(certificate)

['Love and Death', 'Nine Queens', 'The Exterminating Angel', 'The Purple Rose of Cairo', 'The Lost Weekend', 'Sherlock Jr.', 'Sanjuro', 'Elite Squad 2: The Enemy Within', 'Veer-Zaara', 'In America', 'Hera Pheri', 'White Heat', 'Le Dîner de Cons', "Knockin' on Heaven's Door", 'The Virgin Spring', 'The Magdalene Sisters', 'Fireworks', 'Hiroshima Mon Amour', 'The Visitor', 'Jab We Met', 'La Grande Illusion', 'Vampire Hunter D: Bloodlust', 'The Killer', 'Baby', 'Black', 'Andaz Apna Apna', 'Dev.D', 'Special 26', 'Zelig', 'Paan Singh Tomar', 'English Vinglish', 'A Night at the Opera', 'C.R.A.Z.Y.', 'Lifeboat', 'The Rules of the Game', 'Tae Guk Gi: The Brotherhood of War', 'Udta Punjab', 'A Wednesday', 'The Edge of Heaven', 'Munna Bhai M.B.B.S.', 'Chak De! India', 'Vizontele', 'The Circus', 'Kai Po Che', 'OMG: Oh My God!', 'Drishyam', 'Airlift', 'Vicky Donor', 'Lage Raho Munna Bhai', 'The Breath', 'Soul', "It's a Wonderful Life", 'Home Alone', 'A Christmas Story', 'Die Hard', 'Klaus', 'The So

In [17]:
movies = pd.DataFrame({
'movie': titles,
'year': years,
'timeMin': time,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'us_grossMillions': us_gross,
'movie_certificate':certificate
})

In [27]:
movies

,movie,year,timeMin,imdb,metascore,votes,us_grossMillions,movie_certificate
0,Love and Death,1975,85,7.7,89.0,35851,NaN,PG
1,Nine Queens,2000,114,7.9,80.0,49480,1.22,R
2,The Exterminating Angel,1962,95,8.1,NaN,29519,NaN,Not Rated
3,The Purple Rose of Cairo,1985,82,7.7,75.0,46886,10.63,PG
4,The Lost Weekend,1945,101,7.9,NaN,33368,9.46,Passed
...,...,...,...,...,...,...,...,...
1045,Drishyam,2013,160,8.3,NaN,30511,NaN,Not Rated
1046,Airlift,2016,130,8.0,NaN,52667,NaN,Not Rated
1047,Vicky Donor,2012,126,7.8,NaN,39510,0.17,Not Rated
1048,Lage Raho Munna Bhai,2006,144,8.1,NaN,42882,2.22,PG-13


In [23]:
#movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
#movies['timeMin'] = movies['timeMin'].str.extract('(\d+)').astype(int)

movies['metascore'] = pd.to_numeric(movies['metascore'],errors='coerce')
movies['votes'] = movies['votes'].str.replace(',', '').astype(int)
movies['us_grossMillions'] = movies['us_grossMillions'].map(lambda x: x.lstrip('$').rstrip('M'))

movies['us_grossMillions'] = pd.to_numeric(movies['us_grossMillions'], errors='coerce')

In [26]:
movies['movie_certificate']=movies['movie_certificate'].replace('-',np.nan)

In [28]:
movies['movie_certificate'].isnull().sum()

13

In [31]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie              1050 non-null   object 
 1   year               1050 non-null   int32  
 2   timeMin            1050 non-null   int32  
 3   imdb               1050 non-null   float64
 4   metascore          861 non-null    float64
 5   votes              1050 non-null   int32  
 6   us_grossMillions   864 non-null    float64
 7   movie_certificate  1037 non-null   object 
dtypes: float64(3), int32(3), object(2)
memory usage: 53.4+ KB


In [33]:
movies.dtypes

movie                 object
year                   int32
timeMin                int32
imdb                 float64
metascore            float64
votes                  int32
us_grossMillions     float64
movie_certificate     object
dtype: object

In [34]:
movies.to_csv("movies.csv")